In [2]:
from agentic_designs.tool_des.tool import tool
from agentic_designs.tool_des.tool_agent_des import ToolAgent

In [3]:
import requests
from geopy.geocoders import Nominatim

def fetch_osm_restaurants(hotspot_name: str, top_n: int):
    try:
        geolocator = Nominatim(user_agent="smart_osm_agent")
        location = geolocator.geocode(hotspot_name)

        if not location:
            print(f"Location '{hotspot_name}' not found.")
            return []

        lat, lon = location.latitude, location.longitude

        query = f"""
        [out:json];
        (
          node[amenity=restaurant](around:3000,{lat},{lon});
          way[amenity=restaurant](around:3000,{lat},{lon});
          relation[amenity=restaurant](around:3000,{lat},{lon});
        );
        out center;
        """

        url = "https://overpass-api.de/api/interpreter"
        response = requests.post(url, data={"data": query})
        response.raise_for_status()
        data = response.json()

        results = []
        for element in data["elements"]:
            tags = element.get("tags", {})

            if "lat" in element and "lon" in element:
                latitude = element["lat"]
                longitude = element["lon"]
            elif "center" in element:
                latitude = element["center"]["lat"]
                longitude = element["center"]["lon"]
            else:
                continue  

            if tags.get("diet:vegetarian") == "yes" or tags.get("diet:meat") == "no":
                food_type = "Veg"
            elif tags.get("diet:meat") == "only":
                food_type = "Non-Veg"
            else:
                food_type = "Unspecified"

            address_parts = [
                tags.get("addr:housenumber"),
                tags.get("addr:street"),
                tags.get("addr:suburb"),
                tags.get("addr:city"),
                tags.get("addr:state"),
                tags.get("addr:postcode"),
                tags.get("addr:country")
            ]
            full_address = ", ".join([part for part in address_parts if part]) or "Address not available"

            score = sum([
                1 if tags.get("website") else 0,
                1 if tags.get("phone") else 0,
                1 if tags.get("opening_hours") else 0,
                1 if full_address != "Address not available" else 0,
            ])

            results.append({
                "name": tags.get("name", "Unnamed"),
                "address": full_address,
                "phone": tags.get("phone", "Not available"),
                "website": tags.get("website", "Not available"),
                "opening_hours": tags.get("opening_hours", "Not available"),
                "latitude": latitude,
                "longitude": longitude,
                "food_type": food_type
            })
            
        sorted_results = sorted(results, key=lambda x: (
            x["phone"] != "Not available",
            x["website"] != "Not available",
            x["opening_hours"] != "Not available",
            x["address"] != "Address not available"
        ), reverse=True)

        print(f"\n Restaurants near '{hotspot_name}':\n")
        for i, r in enumerate(sorted_results[:top_n], 1):
            print(f" {i}. {r['name']}")
            print(f"    Address: {r['address']}")
            print(f"    Phone: {r['phone']}")
            print(f"    Website: {r['website']}")
            print(f"    Hours: {r['opening_hours']}")
            print(f"    Location: ({r['latitude']}, {r['longitude']})")
            print(f"    Type: {r['food_type']}")
            print("")

        return sorted_results[:top_n]

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []


In [4]:
output = fetch_osm_restaurants("Shibuya, Tokyo", 5)


 Restaurants near 'Shibuya, Tokyo':

 1. かぶいて候
    Address: 5, Roppongi-dori, 106-0031
    Phone: +81 3 3478 1270
    Website: http://www.lemaghrebchandelier.com
    Hours: Tu-Su 11:30-23:00
    Location: (35.6601102, 139.72416)
    Type: Unspecified

 2. 千里眼
    Address: 8, 目黒区, 153-0041
    Phone: +81-3-3481-5773
    Website: https://senrigaaan.com/
    Hours: 11:00-15:00, 17:00-21:00; We off
    Location: (35.6640165, 139.6785021)
    Type: Unspecified

 3. 食堂コマニ DINING LABO
    Address: 1, 目黒区, 153-8505
    Phone: +81-3-6416-8722
    Website: https://shokudo-komani.com
    Hours: 10:00
    Location: (35.661096, 139.677072)
    Type: Unspecified

 4. 菱田屋酒場
    Address: 101, 目黒区
    Phone: +81-50-3743-7433
    Website: https://www.instagram.com/hishidaya_sakaba/
    Hours: Mo-Su 17:00-22:00 (L.O.21:00)
    Location: (35.6575661, 139.6867033)
    Type: Unspecified

 5. マジックスパイス東京店
    Address: 15, 世田谷区北沢, 世田谷区, 155-0031
    Phone: +81-3-5454-8801
    Website: http://magicspice.net
  

In [5]:
restaurant_tool = tool(fetch_osm_restaurants)
restaurant_tool_agent = ToolAgent(tools=[restaurant_tool])
output = restaurant_tool_agent.run(user_msg="Give me the top 5 restaurants in Shibuya, Tokyo")
print(output)


Using Tool: fetch_osm_restaurants

Tool call dict: 
{'name': 'fetch_osm_restaurants', 'arguments': {'hotspot_name': 'Shibuya, Tokyo', 'top_n': 5}, 'id': 1}

 Restaurants near 'Shibuya, Tokyo':

 1. かぶいて候
    Address: 5, Roppongi-dori, 106-0031
    Phone: +81 3 3478 1270
    Website: http://www.lemaghrebchandelier.com
    Hours: Tu-Su 11:30-23:00
    Location: (35.6601102, 139.72416)
    Type: Unspecified

 2. 千里眼
    Address: 8, 目黒区, 153-0041
    Phone: +81-3-3481-5773
    Website: https://senrigaaan.com/
    Hours: 11:00-15:00, 17:00-21:00; We off
    Location: (35.6640165, 139.6785021)
    Type: Unspecified

 3. 食堂コマニ DINING LABO
    Address: 1, 目黒区, 153-8505
    Phone: +81-3-6416-8722
    Website: https://shokudo-komani.com
    Hours: 10:00
    Location: (35.661096, 139.677072)
    Type: Unspecified

 4. 菱田屋酒場
    Address: 101, 目黒区
    Phone: +81-50-3743-7433
    Website: https://www.instagram.com/hishidaya_sakaba/
    Hours: Mo-Su 17:00-22:00 (L.O.21:00)
    Location: (35.6575661,

In [6]:
output = restaurant_tool_agent.run(user_msg="Give me the top 5 restaurants in Soho, London")
print(output)


Using Tool: fetch_osm_restaurants

Tool call dict: 
{'name': 'fetch_osm_restaurants', 'arguments': {'hotspot_name': 'Soho, London', 'top_n': 5}, 'id': 1}

 Restaurants near 'Soho, London':

 1. Giraffe
    Address: Riverside Level 1, London, SE1 8XX
    Phone: +44 20 7042 6900
    Website: https://www.giraffe.net/
    Hours: Mo-Th 08:00-23:00;Fr 08:00-23:30;Sa 09:00-23:30;Su 09:00-22:30
    Location: (51.5062917, -0.1170999)
    Type: Unspecified

 2. Honest Burgers
    Address: 9, Festival Terrace, London, SE1 8XX
    Phone: +44 20 3146 6379
    Website: https://www.honestburgers.co.uk/locations/south-bank/
    Hours: Mo-Tu 12:00-21:00; We-Th 12:00-22:00; Fr-Sa 12:00-22:30; Su 12:00-21:30
    Location: (51.5052621, -0.1167109)
    Type: Veg

 3. Côte Brasserie
    Address: Belvedere Road, London, SE1 8XX
    Phone: +44 20 3846 5606
    Website: https://www.cote.co.uk/restaurant/royal-festival-hall/
    Hours: Mo-Fr 09:00-24:00; Sa 10:00-24:00; Su 10:00-22:00
    Location: (51.5053303